<a href="https://colab.research.google.com/github/MasrourTawfik/DFMEA-LLM-Enhanced/blob/main/Documentation/colabs/affinage_par_ludwig_notebook_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Affinage par LUDWIG


![LUDWIG framework](https://github.com/MasrourTawfik/DFMEA-LLM-Enhanced/blob/main/Documentation/images/Ludwig.png?raw=1)


Cette section explique comment effectuer le fine-tuning d'un modèle de langage en utilisant Ludwig, un outil puissant qui simplifie le cycle de vie de l'apprentissage automatique. Le processus implique l'utilisation d'une configuration de modèle spécifique et d'un ensemble de données pour adapter un modèle pré-entraîné à une tâche spécialisée.

## Configuration

Avant de commencer, assurez-vous que Ludwig est installé et configuré dans votre environnement. Vous devrez également configurer le token du Hugging Face Hub pour accéder aux modèles pré-entraînés. Ceci est réalisé en définissant la variable d'environnement HUGGING_FACE_HUB_TOKEN avec votre clé API Hugging Face :

Ludwig nécessite une configuration YAML qui décrit les paramètres du modèle et de l'entraînement. Les éléments clés de cette configuration incluent :

## Dataset

Le dataset qu'on a utilisé dans cet exemple c'est Alpaca, c'est une ressource unique destinée à l'entraînement et au fine-tuning des modèles de langue pour mieux suivre **les instructions**.

[Alpaca dataset](https://huggingface.co/datasets/tatsu-lab/alpaca)

![DATASET Alpaca](https://github.com/MasrourTawfik/DFMEA-LLM-Enhanced/blob/main/Documentation/images/alpaca.png?raw=1)

## Entraînement

Pour entraîner le modèle, chargez la configuration et spécifiez l'ensemble de données. La classe LudwigModel est utilisée avec la méthode train :

## Sauvegarde du Modèle

Après l'entraînement, sauvegardez le modèle pour une utilisation ou un déploiement ultérieurs :

In [ ]:
!pip install ludwig ludwig[llm] peft

In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = os.getenv('HUGGINGFACE_API_KEY')

In [ ]:
# Ludwig configuration
config_str = """
model_type: llm
base_model: mistralai/Mistral-7B-v0.1
# base_model: alexsherstinsky/Mistral-7B-v0.1-sharded
# base_model: Siddharthvij10/MistralSharded2
quantization:
  bits: 4
adapter:
  type: lora
prompt:
  template: |
    ### Instruction:
    {instruction}
    ### Input:
    {input}
    ### Response:
input_features:
  - name: prompt
    type: text
    preprocessing:
      max_sequence_length: 256
output_features:
  - name: output
    type: text
    preprocessing:
      max_sequence_length: 256
trainer:
  type: finetune
  learning_rate: 0.0001
  batch_size: 1
  gradient_accumulation_steps: 16
  epochs: 3
  learning_rate_scheduler:
    warmup_fraction: 0.01
preprocessing:
  sample_ratio: 0.1
"""

In [ ]:
from ludwig.api import LudwigModel
import yaml
import logging


config = yaml.safe_load(config_str)
model = LudwigModel(config=config, logging_level=logging.INFO)
results = model.train(dataset="ludwig://alpaca")

In [ ]:
model.save("results")

In [ ]:
!python -m ludwig.upload hf_hub --repo_id "your_repo_id" --model_path "path_to_your_model"